In [1]:
"""
Working with Election Office NI Data

The bits that we can; this module is primarily concerned with the ingestion of NI Assembly election results from 2003
onwards (where possible in a vaguely reliable automated way)

Hitlist:
[X] 2022
[X] 2017
[X] 2016
[ ] 2011
[ ] 2007
[ ] 2003

"""
import datetime
import re
from io import BytesIO
from typing import AnyStr
from typing import Dict
from typing import Iterable
from typing import Optional
from typing import Union

import pandas as pd
import requests
from bs4 import BeautifulSoup

ua = f"@Bolster/34ish (+http://bolster.online/)"

def get_excel_dataframe(file_url, requests_kwargs=None, read_kwargs=None):
    if requests_kwargs is None:
        requests_kwargs = {}
    if read_kwargs is None:
        read_kwargs = {}

    with requests.get(file_url, **requests_kwargs) as response:
        response.raise_for_status()
        data = BytesIO(response.content)
        df = pd.read_excel(data, **read_kwargs)
        return df
#
_headers = {
    "user-agent": f"User-Agent: {ua} Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, "
    f"like Gecko) Chrome/91.0.4472.114 Safari/537.36"
}
_base_url = "https://www.eoni.org.uk"


def get_page(path: AnyStr) -> BeautifulSoup:
    """
    For a given path (within EONI.org.uk), get the response as a BeautifulSoup instance

    Note:
        EONI is trying to block people from scraping and will return a 403 error if you don't pass a 'conventional' user agent

    >>> page = get_page("/Elections/")
    >>> page.find('title').contents[0].strip()
    'The Electoral Office of Northern Ireland - EONI'

    """
    res = requests.get(_base_url + path, headers=_headers)
    res.raise_for_status()
    page = BeautifulSoup(res.content, features="html.parser")
    return page


def find_xls_links_in_page(page: BeautifulSoup) -> Iterable[AnyStr]:
    """
    Walk through a BeautifulSoup page and iterate through '(XLS)' suffixed links

    (Primarily Used for 'Results' pages within given elections)

    #WTF Was starting to do some consistency checks between elections to make sure all is kosher, and was wondering why I had a Strangford listing in 2017 but not 2022;
    # As a cross-check on the result page, I walk the links in the right colum of the page, looking for links that have text that ends (XLS). Pretty simple you might think. Except the Strangford link ends in (XLS  and then a random closing ) text string is added to the end.

    >>> page = get_page("/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/Elections-2022/NI-Assembly-Election-2022-Result-Sheets")
    >>> len(list(find_xls_links_in_page(page)))
    18
    >>> next(find_xls_links_in_page(page))
    'https://www.eoni.org.uk/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS'

    """
    for _p in page.select(".right-column a"):
        if "XLS" in _p.contents[0]:
            yield _base_url + _p.attrs["href"]


def normalise_constituencies(cons_str: str) -> str:
    """
    Some constituencies change names or cases etc;

    Use this function to take external/unconventional inputs and project them into a normalised format

    >>> normalise_constituencies('Newry & Armagh')
    'newry and armagh'

    """
    return cons_str.lower().replace(" & ", " and ")


def get_metadata_from_df(
    df: pd.DataFrame,
) -> Dict[str, Union[int, str, datetime.datetime]]:
    """
    Extract Ballot metadata from the table header(s) of an XLS formatted result sheet, as output from `get_excel_dataframe`

    # TODO this could probably be done better as a `dataclass`

    Returns:
        dict of
            'stage': int,
            'date': datetime
            'constituency': str (lower)
            'eligible_electorate': int
            'votes_polled': int
            'number_to_be_elected': int
            'invalid_votes': int
            'electoral_quota': int
    """

    stage_n_catcher = re.compile(r"^Stage (\d+)")

    metadata = {
        "stage": int(re.match(stage_n_catcher, df.columns[5]).group(1)),
        # should have been just int(df.columns[5].split()[-1])., but someone insisted on messing up 2017
        "date": df.columns[10],
        "constituency": normalise_constituencies(df.iloc[0, 3]),
        "eligible_electorate": int(df.iloc[1, 3]),
        "votes_polled": int(df.iloc[2, 3]),
        "number_to_be_elected": int(df.iloc[1, 6]),
        "total_valid_votes": int(df.iloc[2, 6]),
        "invalid_votes": int(df.iloc[1, 9]),
        "electoral_quota": int(df.iloc[1, 12]),
    }
    return metadata


def get_candidates_from_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extract Candidates name and party columns from first stage sheet
    """
    candidates_df = df.iloc[9:29, 2:4]
    candidates_df.columns = ["candidate_name", "candidate_party"]
    return candidates_df.replace(0, None).dropna().reset_index(drop=True)


def get_stage_votes_from_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the votes from each stage as a mapped column for each stage, i.e. stages 1...N
    """
    stages = get_metadata_from_df(df)["stage"]
    stage_df = (
        pd.concat({n: extract_stage_n_votes(df, n) for n in range(stages)})
        .unstack()
        .T.replace(0, None)
        .dropna(how="all")
    )
    return stage_df


def get_stage_transfers_from_df(df):
    """
    Extract the transfers from each stage as a mapped column for each stage, i.e. stages 2...N
    """
    stages = get_metadata_from_df(df)["stage"]
    stage_df = (
        pd.concat({n: extract_stage_n_transfers(df, n) for n in range(stages)})
        .unstack()
        .T.replace(0, None)
        .dropna(how="all") # fixme this drops valid candidate rows where candidates never gave or received transfers (2022, newry)
    )
    return stage_df


def extract_stage_n_votes(df: pd.DataFrame, n: int) -> Optional[pd.Series]:
    """
    Extract the votes from a given stage N

    Note: This will include trailing, unaligned `Nones` which must be cleaned up at the Ballot level
    """
    if n == 0:
        return None
    if n < 10:
        row_offset = 9
        col_offset = 4 + (2 * (n - 1))
    else:
        row_offset = 55
        col_offset = 6 + (2 * (n - 10))

    return df.iloc[row_offset : row_offset + 20, col_offset].reset_index(drop=True)


def extract_stage_n_transfers(df: pd.DataFrame, n: int) -> Optional[pd.Series]:
    """
    Extract the votes from a given stage N

    Note: This will include trailing, unaligned `Nones` which must be cleaned up at the Ballot level
    Stage Transfers are associated with the 'next' stage, i.e. stage 1 has no transfers
    """
    if n <= 1:
        return None
    if n < 10:
        row_offset = 9
        col_offset = 5 + (2 * (n - 2))
    else:
        row_offset = 55
        col_offset = 5 + (2 * (n - 10))

    return df.iloc[row_offset : row_offset + 20, col_offset].reset_index(drop=True)


def get_results_from_sheet(sheet_url: AnyStr) -> Dict[str, Union[pd.DataFrame, dict]]:
    df = get_excel_dataframe(sheet_url, requests_kwargs={"headers": _headers})
    metadata = get_metadata_from_df(df)
    candidates = get_candidates_from_df(df)
    stage_votes = get_stage_votes_from_df(df)
    stage_transfers = get_stage_transfers_from_df(df)

    return {
        "candidates": candidates,
        "stage_votes": stage_votes,
        "stage_transfers": stage_transfers,
        "metadata": metadata,
    }


def get_results(year: int) -> Dict[str, Union[pd.DataFrame, dict]]:
    results_listing_dir = "/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/"
    results_listing_path = {
        2022: "Elections-2022/NI-Assembly-Election-2022-Result-Sheets",
        2017: "Elections-2017/NI-Assembly-Election-2017-Result-Sheets",
        2016: "Elections-2016/NI-Assembly-Election-2016-Candidates-Elected-(1)",
    }

    results = {}
    results_listing_page = get_page(results_listing_dir + results_listing_path[year])
    for sheet_url in find_xls_links_in_page(results_listing_page):
        data = get_results_from_sheet(sheet_url)
        results[data["metadata"]["constituency"]] = data
    return results


In [2]:
results = get_results(2022)

In [3]:
results.keys()

dict_keys(['belfast east', 'belfast north', 'belfast south', 'belfast west', 'east antrim', 'east londonderry', 'fermanagh and south tyrone', 'foyle', 'lagan valley', 'mid ulster', 'newry and armagh', 'north antrim', 'north down', 'south antrim', 'south down', 'strangford', 'upper bann', 'west tyrone'])

In [4]:
results['strangford']['stage_votes']

,1,2,3,4,5,6,7,8,9
0,7015,7015,7015,7015,7015,7015,7015,6811,6811
1,5186,5233,5240,5327,5502,5570.31,5866.41,5868.35,5924.01
2,4704,4728,4737,4788,6140,7488.38,6811,6811,6811
3,2440,2563,3715,3759,3772,3777.94,3780.74,3801.68,None
4,118,None,None,None,None,None,None,None,None
5,831,None,None,None,None,None,None,None,None
6,2822,3279,3568,3670,3697,3706.9,3734.9,3842.16,6173.78
7,1607,1644,None,None,None,None,None,None,None
8,6601,6643,6645,6779,8344,6811,6811,6811,6811
9,3693,3763,3775,5822,6034,6124.09,6471.29,6478.29,6706.89


In [5]:
election_years = [2016,2017,2022]
for e in election_years:
    try:
        results = get_results(e)
    except Exception as exp:
        print(f"Failed on {e}: {exp}")

In [6]:
results.keys()

dict_keys(['belfast east', 'belfast north', 'belfast south', 'belfast west', 'east antrim', 'east londonderry', 'fermanagh and south tyrone', 'foyle', 'lagan valley', 'mid ulster', 'newry and armagh', 'north antrim', 'north down', 'south antrim', 'south down', 'strangford', 'upper bann', 'west tyrone'])

In [7]:
results['belfast east']['stage_votes']

,1,2,3,4,5,6,7,8,9,10,11
0,5281,5315.2,5317.92,5317.92,5346.12,5373.84,5636.68,6691.24,6706.96,6717.96,7628.96
1,970,973.36,974.36,974.36,976.48,991.72,None,None,None,None,None
2,6633,6645.72,6645.84,6645.84,6655.96,6666.32,6995.8,7078.0,7091.24,7092.24,8961.24
3,7253,7253,7253,7209,7209,7209,7209,7209,7209,7209,7209
4,484,507.4,516.64,516.64,None,None,None,None,None,None,None
5,500,509.96,534.2,534.2,576.76,None,None,None,None,None,None
6,1282,1297.48,1298.84,1298.84,1324.56,1339.92,1405.92,None,None,None,None
7,8195,7209,7209,7209,7209,7209,7209,7209,7209,7209,7209
8,72,75.6,None,None,None,None,None,None,None,None,None
9,5820,6611.04,6623.88,6623.88,6811.8,6955.96,6974.32,7071.92,7413.92,7209,7209


In [8]:
results['belfast east']['stage_transfers']

,2,3,4,5,6,7,8,9,10,11
0,34.2,2.72,None,28.2,27.72,262.84,1054.56,15.72,11,911
1,3.36,1,None,2.12,15.24,-991.72,None,None,None,None
2,12.72,0.12,None,10.12,10.36,329.48,82.2,13.24,1,1869
3,None,None,-44,None,None,None,None,None,None,None
4,23.4,9.24,None,-516.64,None,None,None,None,None,None
5,9.96,24.24,None,42.56,-576.76,None,None,None,None,None
6,15.48,1.36,None,25.72,15.36,66,-1405.92,None,None,None
7,-986,None,None,None,None,None,None,None,None,None
8,3.6,-75.6,None,None,None,None,None,None,None,None
9,791.04,12.84,None,187.92,144.16,18.36,97.6,342,-204.92,None


In [9]:
results['belfast east']['stage_transfers'].to_json()

'{"2":{"0":34.2,"1":3.36,"2":12.72,"3":null,"4":23.4,"5":9.96,"6":15.48,"7":-986,"8":3.6,"9":791.04,"10":15.84,"11":3.48,"12":57.72},"3":{"0":2.72,"1":1,"2":0.12,"3":null,"4":9.24,"5":24.24,"6":1.36,"7":null,"8":-75.6,"9":12.84,"10":5.12,"11":2,"12":14.84},"4":{"0":null,"1":null,"2":null,"3":-44,"4":null,"5":null,"6":null,"7":null,"8":null,"9":null,"10":null,"11":null,"12":null},"5":{"0":28.2,"1":2.12,"2":10.12,"3":null,"4":-516.64,"5":42.56,"6":25.72,"7":null,"8":null,"9":187.92,"10":73.4,"11":3,"12":113.56},"6":{"0":27.72,"1":15.24,"2":10.36,"3":null,"4":null,"5":-576.76,"6":15.36,"7":null,"8":null,"9":144.16,"10":37.48,"11":12.12,"12":272.48},"7":{"0":262.84,"1":-991.72,"2":329.48,"3":null,"4":null,"5":null,"6":66,"7":null,"8":null,"9":18.36,"10":3.12,"11":246.12,"12":26.6},"8":{"0":1054.56,"1":null,"2":82.2,"3":null,"4":null,"5":null,"6":-1405.92,"7":null,"8":null,"9":97.6,"10":5,"11":62.24,"12":51.92},"9":{"0":15.72,"1":null,"2":13.24,"3":null,"4":null,"5":null,"6":null,"7":null,"

In [10]:
results['belfast east']['metadata']

{'stage': 12,
 'date': datetime.datetime(2022, 5, 5, 0, 0),
 'constituency': 'belfast east',
 'eligible_electorate': 70123,
 'votes_polled': 43840,
 'number_to_be_elected': 5,
 'total_valid_votes': 43248,
 'invalid_votes': 592,
 'electoral_quota': 7209}

In [16]:
_df = results['belfast east']['stage_votes'].astype(float)

In [17]:
_df.set_index(results['belfast east']['candidates']['candidate_name'])

,1,2,3,4,5,6,7,8,9,10,11
candidate_name,,,,,,,,,,,
"ALLEN, Andy",5281.0,5315.20,5317.92,5317.92,5346.12,5373.84,5636.68,6691.24,6706.96,6717.96,7628.96
"BENNETT, Karl George",970.0,973.36,974.36,974.36,976.48,991.72,NaN,NaN,NaN,NaN,NaN
"BROOKS, David",6633.0,6645.72,6645.84,6645.84,6655.96,6666.32,6995.80,7078.00,7091.24,7092.24,8961.24
"BUNTING, Joanne",7253.0,7253.00,7253.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00
"CARSON, Charlotte",484.0,507.40,516.64,516.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"KENNY, Hannah",500.0,509.96,534.20,534.20,576.76,NaN,NaN,NaN,NaN,NaN,NaN
"KERR, Lauren Alana",1282.0,1297.48,1298.84,1298.84,1324.56,1339.92,1405.92,NaN,NaN,NaN,NaN
"LONG, Naomi",8195.0,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00,7209.00
"MACNEILL, Eoin",72.0,75.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
def plot_constituency_transfers(constituency='belfast east'):
    results[constituency]['stage_transfers'].astype(float)\
             .set_index(results[constituency]['candidates']['candidate_name'])\
             .style.format('{:.0f}', na_rep="")\
             .bar(height=50, align='zero', cmap="bwr",
                  vmin=-results[constituency]['metadata']['electoral_quota']*.5, 
                  vmax=results[constituency]['metadata']['electoral_quota']*.5,
                  width=60)

In [34]:
for constituency in results.keys():
    print(constituency)
    plot_constituency_transfers(constituency)

belfast east
belfast north
belfast south
belfast west
east antrim
east londonderry
fermanagh and south tyrone
foyle
lagan valley
mid ulster
newry and armagh


ValueError: Length mismatch: Expected 10 rows, received array of length 12

In [35]:
results['newry and armagh']

{'candidates':            candidate_name                          candidate_party
 0          BOYLAN, Cathal                                Sinn Féin
 1           COADE, Jackie                           Alliance Party
 2        CONNOLLY, Daniel  Aontú for Life, Unity, Economic Justice
 3           GRANT, Nicola                        The Workers Party
 4            HENRY, Ciara             Green Party Northern Ireland
 5   IRWIN, William George       Democratic Unionist Party - D.U.P.
 6            KIMMINS, Liz                                Sinn Féin
 7           MALONE, Gavin                              Independent
 8         MCNULTY, Justin  SDLP (Social Democratic & Labour Party)
 9           MURPHY, Conor                                Sinn Féin
 10       RATCLIFFE, Keith                      TUV - No Sea Border
 11   TAYLOR, David Samuel                    Ulster Unionist Party,
 'stage_votes':        1     2     3     4
 0   9843  9843  9843  9843
 1   3345  3637  4088  4741
 2